In [1]:
import io
import time
import json
import datetime as dt
import csv
import random
from random import randint
import numpy as np 
import pandas as pd 
from faker import Faker
from faker.providers import BaseProvider
from sdg_ml.config.sdg_conf import SdgConf
from sdg_ml.connector.s3_connector import S3Connector

In [2]:
import warnings
warnings.filterwarnings("ignore")

### S3 Connector Conf

In [3]:
sdg_conf = SdgConf("/home/cloud-user/a495587/scripts/s3_connector/test_config_s3.json")

In [4]:
s3_connector = S3Connector(sdg_conf)

### Read Data

In [5]:
def read_parquet(s3_connector, path):
    s3_client = s3_connector.get_client()
    object_as_bytes = s3_client.get_object(Bucket=s3_connector._sdg_conf._s3_bucket, Key=path)
    return pd.read_parquet(io.BytesIO(object_as_bytes['Body'].read()))

In [6]:
def read_csv_clean(s3_connector, path):
    s3_client = s3_connector.get_client()
    object_as_bytes = s3_client.get_object(Bucket=s3_connector._sdg_conf._s3_bucket, Key=path)
    return pd.read_csv(io.BytesIO(object_as_bytes['Body'].read()), escapechar = '\\', on_bad_lines='skip', quoting=csv.QUOTE_NONE, engine ='python')

In [7]:
def read_json(s3_connector, path):
    s3_client = s3_connector.get_client()
    object_as_bytes = s3_client.get_object(Bucket=s3_connector._sdg_conf._s3_bucket, Key=path)
    return pd.read_json(io.BytesIO(object_as_bytes['Body'].read()), lines = True)

### Get Multi Data

In [8]:
tracking_id = 'gtps_fccr_model'
original_path = f"a495587/original_data/original_{tracking_id}/"
original_file_list = s3_connector.list_objects(original_path)
list_format = ['orc', 'parquet']

In [9]:
real_data_dict = {}
for format in list_format:
    list_file = list(filter(lambda a : format in a, original_file_list))
    for data_path in list_file:
        print(data_path)
        print(data_path.split('/')[3])
        table_name = data_path.split('/')[3]
        if format == 'orc':
            data = s3_connector.read_orc(data_path)
        elif format == 'parquet':
            data = read_parquet(s3_connector, data_path)
        
        real_data_dict[table_name] = data

a495587/original_data/original_gtps_fccr_model/bd_dd_remittance_paidin/data/int_bd_dd_remittance_paidin.orc
bd_dd_remittance_paidin
a495587/original_data/original_gtps_fccr_model/bd_party_details_paidin/data/int_bd_party_details_paidin.orc
bd_party_details_paidin
a495587/original_data/original_gtps_fccr_model/bd_party_details_paidout/data/int_bd_party_details_paid_out.orc
bd_party_details_paidout
a495587/original_data/original_gtps_fccr_model/bd_account/data/bd_account.parquet
bd_account
a495587/original_data/original_gtps_fccr_model/bd_bpmainonscreen/data/int_bd_bpmainonscreen.parquet
bd_bpmainonscreen
a495587/original_data/original_gtps_fccr_model/bd_communication_address/data/int_bd_communication_address.parquet
bd_communication_address
a495587/original_data/original_gtps_fccr_model/bd_dd_paidout/data/dd_paid_out.parquet
bd_dd_paidout
a495587/original_data/original_gtps_fccr_model/bd_transfer_remittance/data/transfer_remittance.parquet
bd_transfer_remittance
a495587/original_data/or

In [10]:
for table_name, data in real_data_dict.items():
    print(f'{table_name} shape : {data.shape}')

bd_dd_remittance_paidin shape : (4771903, 134)
bd_party_details_paidin shape : (10441388, 26)
bd_party_details_paidout shape : (10495888, 26)
bd_account shape : (416197, 94)
bd_bpmainonscreen shape : (6968328, 86)
bd_communication_address shape : (1357, 5)
bd_dd_paidout shape : (2547025, 96)
bd_transfer_remittance shape : (1867673, 368)
rd_fccr_clear_system shape : (18, 3)


In [11]:
def drop_fullna(data):
    """ Drop columns with full missing values
    Args:
        data (pandas.DataFrame)
            data 
    Returns:
        data (pandas.DataFrame)
            data without columns with full missing values
    """
    col_to_drop = []
    for col in list(data.columns):
        if data.isnull().sum()[col] == len(data):
            col_to_drop.append(col)
    data = data.drop(columns=col_to_drop)
    return data

### Anonymisation

In [12]:
def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

In [13]:
class CustomProvider:
    @classmethod
    def bdr_id(cls):
        faker = Faker()
        return faker.pystr_format("#########{{random_int}}") 

    @classmethod
    def iban_number(cls):
        faker = Faker()
        fake_iban = faker.country_code() + str(random_with_N_digits(2)) + faker.pystr(max_chars = 4).upper() + str(random_with_N_digits(6)) + str(random_with_N_digits(8))
        return fake_iban.upper() 
    
    @classmethod
    def id_class(cls):
        faker = Faker()
        return faker.pystr_format("#####{{random_int}}") 
    
    @classmethod
    def account_currency(cls):
        faker = Faker()
        return faker.pystr(max_chars = 3).upper()

    @classmethod
    def ten_letters(cls):
        faker = Faker()
        return faker.pystr(max_chars = 10).upper()
    
    @classmethod
    def two_letters(cls):
        faker = Faker()
        return faker.pystr(max_chars = 2).upper()
    
    @classmethod
    def one_letter(cls):
        faker = Faker()
        return faker.pystr(max_chars = 1).upper()
    
    @classmethod
    def num_compte(cls):
        faker = Faker()
        return faker.country_code() + str(random_with_N_digits(25))

In [14]:
def anonymizer(data, list_anonymised_dict):
    """
    Args:
        data (pandas.DataFrame)
            dataframe to anonymise
        list_anonymised_dict (list of dictionary)
            list of dictionary containing key as columns to anonymise & value as Faker category
    Returns:
        data (pandas.DataFrame)
            dataframe anonymised
        mapping (dictionary)
            mapping containing key as original value & value as anonymised value
    """
    mapping = {}
    for anonymised_dict in list_anonymised_dict:
        category = anonymised_dict['category']
        for col in anonymised_dict['col']:
            faker_unique_list = []
            unique_value = list((data[col]).unique())
            faker = Faker()
            faker.add_provider(CustomProvider())
            for i in range(len(unique_value)): 
                faker_unique_list.append(getattr(faker.unique, category)())
            mapping_anonymised = dict(zip(unique_value, faker_unique_list))
            mapping[col] = mapping_anonymised
            data[col] = data[col].map(mapping_anonymised)
    return data, mapping

In [15]:
def anonymizer_multi(data_dict, anonymised_list):
    """
    Args:
        data_dict (dictionary of pandas.DataFrame)
            dictionary of dataframe to anonymise
        anonymised_list (list of dictionary)
            list of dictionary containing key as columns to anonymise & value as Faker category
    Returns:
        data (pandas.DataFrame)
            dataframe anonymised
    """
    mapping_multi = {}
    for table_name in data_dict:
        mapping_multi[table_name] = {}
    for anonymised_dict in anonymised_list:
        faker = Faker()
        faker.add_provider(CustomProvider())        
        for col in anonymised_dict['col']:
            print('col', col)
            unique_value = []
            faker_unique_list = []
            for table_name in anonymised_dict['table_name_list']:
                print('table', table_name)
                unique_value += list((data_dict[table_name][col]).unique())

                for i in range(len(unique_value)): 
                    faker_unique_list.append(getattr(faker.unique, anonymised_dict['category'])())
            
            mapping_anonymised = dict(zip(unique_value, faker_unique_list))
            for table_name in anonymised_dict['table_name_list']:
                mapping_multi[table_name][col] = mapping_anonymised
                data_dict[table_name][col] = data_dict[table_name][col].map(mapping_anonymised)
    return data_dict, mapping_multi 

In [16]:
def anonymizer_iban_prepro(data, list_col):
    """
    Args:
        data (pandas.DataFrame)
        list_col (list of string)
            list of col to anonymize
    Returns:
        data anonymized (pandas.DataFrame)
    """
    for col in list_col:
        print('col', col)
        data[f'{col}_2_letters'] = data[col].str[:2]
        
    return data 

In [17]:
def anonymizer_iban_post(data, list_col):
    """
    Args:
        data (pandas.DataFrame)
        list_col (list of string)
            list of col to anonymize
    Returns:
        data anonymized (pandas.DataFrame)
    """
    for col in list_col:
        print('col', col)
        data[f'{col}_new'] = data[col]
        data = data.drop(columns = [col])
        data[col] = data[f'{col}_2_letters'] + data[f'{col}_new'].str[2:]
        print('Dropping col')
        data = data.drop(columns = [f'{col}_2_letters', f'{col}_new'])
    return data 

### Convert float to int type columns

In [18]:
def clear_float_columns(data):
    """
    Args: 
        data (pandas.DataFrame)
    Returns:
        data with float columns convert to int 
    """
    list_col_float = list(data.select_dtypes(include=['float64']).columns)
    try:
        for col in list_col_float:
            data[col] = data[col].astype('Int64', errors = 'ignore')
            pass
    except:
        pass
    return data

## Columns to Anonymised

**PS :** rajouter le temps d'anonymisation pour chaque table 

In [19]:
real_data_dict.keys()

dict_keys(['bd_dd_remittance_paidin', 'bd_party_details_paidin', 'bd_party_details_paidout', 'bd_account', 'bd_bpmainonscreen', 'bd_communication_address', 'bd_dd_paidout', 'bd_transfer_remittance', 'rd_fccr_clear_system'])

In [20]:
real_data_dict['bd_account'].isnull().sum()

operational_unit         0
pos_id                   0
sub_account_id           0
account_type             0
account_usage            0
                     ...  
van_applcbtly       332755
neg_cr_intrst       332755
neg_dr_intrst       332755
acnt_clsr_rsn       416197
dt                       0
Length: 94, dtype: int64

In [21]:
real_data_dict['bd_party_details_paidout'].shape

(10495888, 26)

In [22]:
real_data_dict['bd_party_details_paidout']['party_adrs_frmt'].isnull().sum()

10495888

### 0) Columns with relationship 

- bd_account.iban_number = remittance_paidin.account
- bd_account.external_reference_1 = remittance_paidin.debtor_account
- bd_account.iban_number = bd_dd_paidout.pos_id

### 1) Int_bd_account 

- bdr_id -> Custom by format
- iban_number -> Country
- account_name -> Company Name 

**No Need:**
- account_currency -> 3 Random Lettersm



### Review 
•	The following columns appear to be floating numbers but should be int: account_manager, deputy_manager, fee_package, van_applcbtly, neg_cr_intrst, neg_dr_intrst, technical_nostro, and maybe multiple others => Tom

•	The following columns appear to be floating numbers but should be strings: bban_bank_code, webclear_account_number, and maybe multiple others => Tom

•	account_description and remark columns seems to contain indentifiable information and should be anonymized => Shivani and Samar please confirm

•	external_reference_2 seems to be an IBAN and should be anonymized => Shivani and Samar please confirm


In [23]:
list_bdr_id_account_anonym = [
'bdr_id',
]

In [24]:
list_company_name_account_anonym = [
'account_name',
]

In [25]:
list_iban_account_anonym = [
#'iban_number',
'external_reference_2', 
#'external_reference_1',
]

In [26]:
list_ten_letters_account_anonym = [
'remark', 
'account_description',
]

In [27]:
bd_account = real_data_dict['bd_account']

In [28]:
list_col_object_bd_account = list(bd_account.select_dtypes(include=['object']).columns)

In [29]:
for col in list_col_object_bd_account:
    bd_account[col] = bd_account[col].replace('nan', None)

In [30]:
bd_account

,operational_unit,pos_id,sub_account_id,account_type,account_usage,account_status,account_currency,product_type,instrument,business_partner_id,...,cti_account_type,entity_code,local_curr_code,root_code,lead_acnt_num,van_applcbtly,neg_cr_intrst,neg_dr_intrst,acnt_clsr_rsn,dt
0,GSOCGZURCH,1327,1,Cash,General,Active,CHF,None,Cash Account - Corporate,154,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,20190403
1,GSOCGZURCH,1557,1,Cash,General,Active,EUR,None,Cash Account - Corporate,214,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,20190403
2,GSOCGZURCH,1558,1,Cash,General,Active,CHF,None,Cash Account - Corporate,214,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,20190403
3,GSOCGZURCH,1425,1,Cash,General,Active,CHF,None,Cash Account - Corporate,315,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,20190403
4,GSOCGZURCH,957,1,Cash,Vostro,Active,CHF,None,Cash Account - Bank,55,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,20190403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416192,GSOCGNLAMS,111950,1,Cash,Vostro,Active,AED,None,Cash Account - Business Line,16736,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,20191023
416193,GSOCGESMAD,2113,1,Cash,Vostro,Active,EUR,None,Cash Account - Bank,2002,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,20191023
416194,GSOCGESMAD,2159,1,Cash,General,Active,EUR,None,Cash Account - Corporate,2015,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,20191023
416195,GSOCGESMAD,12150,1,Cash,General,Active,EUR,None,Cash Account - Corporate,19688,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,20191023


In [31]:
anonymised_dict_account = [{'col': list_iban_account_anonym, 'category': 'iban_number'},
                                  {'col': list_company_name_account_anonym, 'category': 'company'},
                                  {'col': list_bdr_id_account_anonym, 'category': 'bdr_id'},
                                  {'col': list_ten_letters_account_anonym, 'category': 'ten_letters'}
]

In [32]:
bd_account_anonymised = bd_account.copy()

In [33]:
start = time.time()            

In [34]:
bd_account_anonymised = anonymizer_iban_prepro(bd_account_anonymised, list_iban_account_anonym)

col external_reference_2


In [35]:
bd_account_anonymised, mapping = anonymizer(bd_account_anonymised, anonymised_dict_account)

In [36]:
bd_account_anonymised = anonymizer_iban_post(bd_account_anonymised, list_iban_account_anonym)

col external_reference_2
Dropping col


In [37]:
stop = time.time()
print(f'Anonymisation Time {stop - start}s')

Anonymisation Time 587.137444972992s


In [38]:
real_data_dict['bd_account'] = bd_account_anonymised

In [39]:
bd_account_anonymised

,operational_unit,pos_id,sub_account_id,account_type,account_usage,account_status,account_currency,product_type,instrument,business_partner_id,...,entity_code,local_curr_code,root_code,lead_acnt_num,van_applcbtly,neg_cr_intrst,neg_dr_intrst,acnt_clsr_rsn,dt,external_reference_2
0,GSOCGZURCH,1327,1,Cash,General,Active,CHF,None,Cash Account - Corporate,154,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,20190403,CH35ZNXW82861473081530
1,GSOCGZURCH,1557,1,Cash,General,Active,EUR,None,Cash Account - Corporate,214,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,20190403,CH83BCWF65571799457686
2,GSOCGZURCH,1558,1,Cash,General,Active,CHF,None,Cash Account - Corporate,214,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,20190403,CH40IQEQ63167039246315
3,GSOCGZURCH,1425,1,Cash,General,Active,CHF,None,Cash Account - Corporate,315,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,20190403,CH44OBXP43324897213826
4,GSOCGZURCH,957,1,Cash,Vostro,Active,CHF,None,Cash Account - Bank,55,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,20190403,CH42UCHC93666232839951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416192,GSOCGNLAMS,111950,1,Cash,Vostro,Active,AED,None,Cash Account - Business Line,16736,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,20191023,NL61UUSK46616490383997
416193,GSOCGESMAD,2113,1,Cash,Vostro,Active,EUR,None,Cash Account - Bank,2002,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,20191023,ES73PHKU91148940349939
416194,GSOCGESMAD,2159,1,Cash,General,Active,EUR,None,Cash Account - Corporate,2015,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,20191023,ES54JZYS72593036887863
416195,GSOCGESMAD,12150,1,Cash,General,Active,EUR,None,Cash Account - Corporate,19688,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,20191023,ES29VWYF36804389053960


### Filter by bp_id

In [40]:
#bd_account_filter = account_anonymised.copy()
#list_bp_id_filter = [
#17334,
#17336,
#17338,
#16734,
#5996
#]

In [41]:
#mask_bp_id = bd_account_filter['business_partner_id'].isin(list_bp_id_filter)
#bd_account_filter = bd_account_filter[mask_bp_id]

In [42]:
#bd_account_filter = clear_float_columns(bd_account_filter)

In [43]:
#anonymised_path_account_filter = f'a495587/original_data/original_{tracking_id}_anonym_filter/int_bd_account/data/bd_account_anonym.parquet'

In [44]:
#s3_connector.write_parquet(anonymised_path_account_filter, bd_account_filter)

## 2) Int_bd_bpmainonscreen

In [45]:
bpmainonscreen = real_data_dict['bd_bpmainonscreen']

In [46]:
list_company_name_bpmainonscreen_anonym = [
'full_lgl_nm',
'nm_1',
'nm_1_1',
'nm_1_2',
'nm_1_3',
'nm_1_4',
'nm_1_5',
'frst_line_of_prfrd_name', 
'scnd_line_of_prfrd_name',  
'frst_name', 
'mdle_nm', 
'brth_plc', 
'mthr_nm', 
]

In [47]:
list_address_bpmainonscreen_anonym = [
'shrt_adrs1', 
'shrt_adrs2',
]

In [48]:
list_country_code_bpmainonscreen_anonym = ['cntry_of_risk']

In [49]:
list_bdr_id_anonym = ['bdr_id']

In [50]:
list_not_in_bpmainonscreen = [
'nm_1_6',
'nm_2_1',
'nm_2_2',
'nm_2_3',
'nm_2_4',
'nm_2_5',
'nm_2_6',
'idnty_cntry'
]

In [51]:
anonymised_dict_bpmainonscreen = [{'col': list_address_bpmainonscreen_anonym, 'category': 'address'},
                                  {'col': list_country_code_bpmainonscreen_anonym, 'category': 'country_code'},
                                  {'col': list_company_name_bpmainonscreen_anonym, 'category': 'company'},
                                  {'col': list_bdr_id_anonym, 'category': 'bdr_id'},
]

In [52]:
start = time.time()            
bpmainonscreen_anonymised, mapping = anonymizer(bpmainonscreen, anonymised_dict_bpmainonscreen)
stop = time.time()
print(f'Anonymisation Time {stop - start}s')

Anonymisation Time 152.34622883796692s


In [53]:
real_data_dict['bd_bpmainonscreen'] = bpmainonscreen_anonymised

## 3) Int_bd_communication_address

- comm_adrs -> Address

In [54]:
communication_address = real_data_dict['bd_communication_address']

In [55]:
communication_address['comm_adrs']

0       ABSAZAJJAIS
1          RSBNRUMM
2          SEIOGB2L
3          TRDIGB2L
4          SWQBCHZZ
           ...     
1352       ABSAZAJJ
1353       ABNABE2A
1354       DEIBEGCX
1355       RSBNRUMM
1356    SOGEFRPPSGO
Name: comm_adrs, Length: 1357, dtype: object

In [56]:
anonymised_dict_communication_address = [{'col': ['comm_adrs'], 'category': 'address'}]

In [57]:
start = time.time()            
communication_address_anonymised, mapping = anonymizer(communication_address, anonymised_dict_communication_address)
stop = time.time()
print(f'Anonymisation Time {stop - start}s')

Anonymisation Time 0.31684231758117676s


In [58]:
real_data_dict['bd_communication_address'] = communication_address_anonymised

## 4) Int_bd_transfer_remittance anonymiser

### Specific Rules

**IBAN**: 
- credit_transfer_credit_account_reference

**10 Letters**:

- credit_transfer_settlement_remarks_line1 
- credit_transfer_settlement_remarks_line2 
- credit_transfer_settlement_remarks_line3 
- credit_transfer_settlement_remarks_line4 


**Also:**

- credit_transfer_debit_bp_name
- credit_transfer_debit_correspondent_bp_name
- credit_transfer_correspondent_bp_name 

In [59]:
transfer_remittance = real_data_dict['bd_transfer_remittance']

In [60]:
transfer_remittance.shape

(1867673, 368)

### Filter by Date 2022

In [61]:
transfer_remittance['credit_transfer_debit_execution_date'] = pd.to_datetime(transfer_remittance['credit_transfer_debit_execution_date'])
mask = transfer_remittance['credit_transfer_debit_execution_date'].dt.year == int(2022)
transfer_remittance_2022 = transfer_remittance[mask]

In [62]:
transfer_remittance_2022['credit_transfer_debit_execution_date'] = transfer_remittance_2022['credit_transfer_debit_execution_date'].dt.date

In [63]:
transfer_remittance_2022.shape

(1867673, 368)

#### List to anonymised per type

In [64]:
list_address_transfer = [    
'credit_transfer_third_party_address_line3',  
'credit_transfer_third_party_address_line2',  
'credit_transfer_intermadry_address_line1',  
'credit_transfer_intermadry_address_line2',  
'credit_transfer_intermadry_address_line3',
'credit_transfer_ordering_cust_address_line1',  
'credit_transfer_ordering_cust_address_line2', 
'credit_transfer_ordering_cust_address_line3',  
'credit_transfer_ordering_institution_address_line1',  
'credit_transfer_ordering_institution_address_line2',  
'credit_transfer_ordering_institution_address_line3',  
'credit_transfer_receiver_crspndnt_address_line1',  
'credit_transfer_receiver_crspndnt_address_line2',  
'credit_transfer_receiver_crspndnt_address_line3',
'credit_transfer_beneficiary_party_address_line1', 
'credit_transfer_beneficiary_party_address_line2',  
'credit_transfer_beneficiary_party_address_line3',  
'credit_transfer_intermediary_institution_address_line1',  
'credit_transfer_intermediary_institution_address_line2',  
'credit_transfer_intermediary_institution_address_line3',  
'credit_transfer_cover_details_ordering_institution_address_line1',  
'credit_transfer_cover_details_ordering_institution_address_line2',  
'credit_transfer_cover_details_ordering_institution_address_line3',  
'credit_transfer_cover_beneficiary_bank_address_line1',  
'credit_transfer_cover_beneficiary_bank_address_line2',  
'credit_transfer_cover_beneficiary_bank_address_line3',  
'credit_transfer_ordering_party_adrs_line1',  
'credit_transfer_ordering_party_adrs_line2',  
'credit_transfer_ordering_party_adrs_line3',
'credit_transfer_beneficiary_bank_address_line1',  
'credit_transfer_beneficiary_bank_address_line2',  
'credit_transfer_beneficiary_bank_address_line3',    
'credit_transfer_instr_party_address_line1',  
'credit_transfer_instr_party_address_line2',  
'credit_transfer_instr_party_address_line3',  
'credit_transfer_intermediary_address_line1',  
'credit_transfer_intermediary_address_line2',  
'credit_transfer_intermediary_address_line3',  
'credit_transfer_rcvr_crspndnt_address_line1',  
'credit_transfer_rcvr_crspndnt_address_line2',  
'credit_transfer_rcvr_crspndnt_address_line3',  
'credit_transfer_si_beneficiary_bank_address_line1',  
'credit_transfer_si_beneficiary_bank_address_line2',  
'credit_transfer_si_beneficiary_bank_address_line3',  
'credit_transfer_settlement_account_address_line1',  
'credit_transfer_settlement_account_address_line2',  
'credit_transfer_settlement_account_address_line3',  
'credit_transfer_settlement_account_address_line4',  
'credit_transfer_ulti_creditor_address_line1',  
'credit_transfer_ulti_creditor_address_line2',  
'credit_transfer_ulti_creditor_address_line3']

In [65]:
# No Need
list_currency_transfer = [
'credit_transfer_currency',
'credit_transfer_counter_currency',  
'credit_transfer_account_reference_currency',  
'credit_transfer_debit_actual_crncy',  
'credit_transfer_trf_crncy',  
'credit_transfer_credit_actual_crncy',
'credit_transfer_correspondent_account_currency' 
]

In [66]:
list_company_name_transfer = [
'credit_transfer_settlement_account_name',
'credit_transfer_ordering_cust_name',  
'credit_transfer_beneficiary_bank_account_name',    
'credit_transfer_intermediary_name',  
'credit_transfer_ultimate_creditor_name',
'credit_transfer_intermadry_name',  
'credit_transfer_ordering_institution_name',  
'credit_transfer_receiver_crspndnt_name',
'type_of_booking',
'credit_transfer_beneficiary_party_address_type',  
'credit_transfer_debit_bp_name', 
'credit_transfer_debit_correspondent_bp_name', 
'credit_transfer_correspondent_bp_name'
]

In [67]:
list_iban_transfer = [
'credit_transfer_credit_account_reference'
]

In [68]:
list_two_letter_transfer = ['credit_transfer_cntry_code_ben', 'credit_transfer_ind_ben_name', 'credit_transfer_ind_ben_name_whtlst']

In [69]:
list_ten_letter_transfer = [
'credit_transfer_settlement_remarks_line1', 
'credit_transfer_settlement_remarks_line2', 
'credit_transfer_settlement_remarks_line3',
]

In [70]:
full_na_list_transfer = [
'credit_transfer_cntry_code_ord',
'credit_transfer_ind_ben_name_dummy',
'credit_transfer_ind_ord_cust_name',
'credit_transfer_ind_ord_cust_adrs',
'credit_transfer_third_party_address_line1',
]

In [71]:
anonymised_dict_transfer_remittance = [{'col': list_company_name_transfer, 'category': 'company'}, 
        {'col': list_two_letter_transfer, 'category': 'two_letters'}, 
        {'col': list_address_transfer, 'category': 'address'},
        {'col': list_ten_letter_transfer, 'category': 'ten_letters'},
        {'col': list_iban_transfer, 'category': 'iban_number'}
]

In [72]:
transfer_remittance_2022 = anonymizer_iban_prepro(transfer_remittance_2022, list_iban_transfer)

col credit_transfer_credit_account_reference


In [73]:
start = time.time()            
transfer_remittance_anonymised, mapping = anonymizer(transfer_remittance_2022, anonymised_dict_transfer_remittance)
stop = time.time()
print(f'Anonymisation Time {stop - start}s')

Anonymisation Time 10905.12080836296s


In [74]:
transfer_remittance_anonymised = anonymizer_iban_post(transfer_remittance_anonymised, list_iban_transfer)

col credit_transfer_credit_account_reference
Dropping col


In [75]:
real_data_dict['bd_transfer_remittance'] = transfer_remittance_anonymised

### Relationship

- int_bd_transfer_remittance.operational_unit = fccr.operational_unit
- int_bd_transfer_remittance.credit_transfer_settlement_credit_account_pos_typ = int_bd_account.account_type
- int_bd_transfer_remittance.credit_transfer_settlement_account_pos_num = int_bd_account.pos_id 
- int_bd_account. account_currency = int_bd_transfer_remittance. credit_transfer_credit_actual_crncy
- int_bd_account.business_partner_id = int_bd_bpmainonscreen.business_partner_id 
- int_bd_bpmainonscreen.business_partner_id = int_bd_communication_address.business_partner_id

### 5) bd_dd_paid_in

**company_name:**

- debtor_name
- ind_ord_cust_name
- ind_ben_name
- ind_ben_name_dummy
- booking_type
  
**bdr_id:**
- bdr_id

**address:**

- ind_ben_adrs
- ind_ord_cust_adrs

**country_code:**

- cntry_code_ord
- cntry_code_ben

In [76]:
dd_remittance_paid_in = real_data_dict['bd_dd_remittance_paidin']

In [77]:
list_company_name_dd = [
'debtor_name',
'ind_ord_cust_name',
'ind_ben_name',
'ind_ben_name_dummy',
'booking_type',
]

In [78]:
list_address_name_dd = [
'ind_ben_adrs',
'ind_ord_cust_adrs'
]

In [79]:
list_country_code_dd = [
'cntry_code_ben',
'cntry_code_ord',    
]

In [80]:
anonymised_dict_dd = [{'col': list_country_code_dd, 'category': 'country_code'},
                                  {'col': list_company_name_dd, 'category': 'company'},
                                  {'col': list_address_name_dd, 'category': 'address'},
                                  {'col': ['bdr_id'], 'category': 'bdr_id'}
]

In [81]:
start = time.time()            
dd_remittance_paidin_anonymised, mapping = anonymizer(dd_remittance_paid_in, anonymised_dict_dd)
stop = time.time()
print(f'Anonymisation Time {stop - start}s')

Anonymisation Time 1354.113723039627s


In [82]:
real_data_dict['bd_dd_remittance_paidin'] = dd_remittance_paidin_anonymised

### 6) int_bd_party_details_paid_in

**company_name:**
- party_details_party_name
- party_details_contact_name

**address:**
- party_details_adress_line1
- party_details_adress_line2

**full NA:**
- party_details_party_bldg_nm
- party_details_party_flr
- party_details_party_pst_bx
- party_details_party_room
- party_details_party_twn_lctn_nm
- party_details_party_dstrct_nm

In [83]:
party_details_paidin = real_data_dict['bd_party_details_paidin'] 

In [84]:
list_company_name_party = [
'party_details_party_name',
'party_details_contact_name'
]

In [85]:
list_address_name_party = [
'party_details_adress_line1',
'party_details_adress_line2'
]

In [86]:
anonymised_dict_party_details_paidin = [{'col': list_company_name_party, 'category': 'company'},
                                 {'col': list_address_name_party, 'category': 'address'},
]

In [87]:
start = time.time()            
party_details_paidin_anonymised, mapping = anonymizer(party_details_paidin, anonymised_dict_party_details_paidin)
stop = time.time()
print(f'Anonymisation Time {stop - start}s')

Anonymisation Time 1778.7428843975067s


In [88]:
real_data_dict['bd_party_details_paidin'] = party_details_paidin_anonymised 

### 7) int bd_dd_paid_out

**company_name:**
- creditor_name
- ind_ben_name 

**10 Letters:**
- creditor_account_reference 
- creditor_bank_identifier 
- creditor_account
- reciever_info_line1 
- reciever_info_line2 
 
**2 Letter:**
- ind_ord_cust_account
- ind_ord_cust_name 
- ind_ord_cust_adrs 
- ind_ben_adrs
- ind_ord_name_whtlst
- ind_ben_name_whtlst 

**country_code:**
- cntry_code_ord 
- cntry_code_ben 

**bdr_id:**
- bdr_id

**Full Null Columns:**
- ind_ord_name_dummy 
- ind_ben_name_dummy
- reciever_info_line3
- reciever_info_line4
- reciever_info_line5
- reciever_info_line6

In [89]:
dd_paid_out = real_data_dict['bd_dd_paidout']

In [125]:
list_company_name_dd = [
'creditor_name',
'ind_ben_name'
]

In [ ]:
list_10_letters_dd = [
'creditor_bank_identifier',
'reciever_info_line1',
'reciever_info_line2',
]

In [ ]:
list_2_letters_dd = [
'ind_ord_cust_account',
'ind_ord_cust_name',
'ind_ord_cust_adrs',
'ind_ben_adrs',
'ind_ord_name_whtlst',
'ind_ben_name_whtlst', 
]

In [ ]:
list_country_code_dd = [
'cntry_code_ord',
'cntry_code_ben'
]

In [ ]:
anonymised_dict_dd = [{'col': list_company_name_dd, 'category': 'company'},
                      {'col': list_10_letters_dd, 'category': 'ten_letters'},
                      {'col': list_2_letters_dd, 'category': 'two_letters'},
                      {'col': list_country_code_dd, 'category': 'country_code'},
                      {'col': ['bdr_id'], 'category': 'bdr_id'},
]

In [ ]:
list_iban_paidout = [
'creditor_account',
'creditor_account_reference'
]

In [ ]:
start = time.time()
dd_paidout_anonymised = anonymizer_iban_prepro(dd_paid_out, list_iban_paidout)
stop = time.time()
print(f'Anonymisation IBAN Prepro Time {stop - start}s')

col creditor_account
col creditor_account_reference
Anonymisation IBAN Prepro Time 1.2719454765319824s


In [ ]:
start = time.time()            
dd_paidout_anonymised, mapping = anonymizer(dd_paidout_anonymised, anonymised_dict_dd)
stop = time.time()
print(f'Anonymisation IBAN Post Time {stop - start}s')

Anonymisation IBAN Post Time 75.19976282119751s


In [ ]:
start = time.time()
dd_paidout_anonymised = anonymizer_iban_post(dd_paidout_anonymised, list_iban_paidout)
stop = time.time()
print(f'Anonymisation Time {stop - start}s')

col creditor_account
Dropping col
col creditor_account_reference
Dropping col
Anonymisation Time 12.486927270889282s


In [ ]:
real_data_dict['bd_dd_paidout'] = dd_paidout_anonymised

### 8) int_bd_party_details_paid_out

**company_name:**
- party_name
- party_account_identifier_name

**address:**
- party_adress_line1
- party_adress_line2

**iban:**
- party_identifier_id

**full NA:**
- party_contact_name
- party_bldg_nm
- party_flr
- party_pst_bx
- party_room
- party_twn_lctn_nm
- party_dstrct_nmcount_identifier_name'
]


In [ ]:
party_details_paidout = real_data_dict['bd_party_details_paidout']

In [ ]:
list_company_name_party = [
'party_name',
'party_account_identifier_name'
]

In [ ]:
list_address_name_party = [
'party_adress_line1',
'party_adress_line2'
]

In [ ]:
list_iban_party = [
'party_identifier_id'
]

In [ ]:
anonymised_dict_party_details = [{'col': list_company_name_party, 'category': 'company'},
                                 {'col': list_address_name_party, 'category': 'address'},
                                 {'col': list_iban_party, 'category': 'iban_number'},
]

In [ ]:
start = time.time()            
party_details_paidout = anonymizer_iban_prepro(party_details_paidout, list_iban_party)
stop = time.time()
print(f'Anonymisation IBAN Prepro Time {stop - start}s')

col party_identifier_id
Anonymisation IBAN Prepro Time 1.9297809600830078s


In [ ]:
start = time.time() 
party_details_paidout_anonymised, mapping = anonymizer(party_details_paidout, anonymised_dict_party_details)
stop = time.time()
print(f'Anonymisation Time {stop - start}s')

Anonymisation Time 164.5794014930725s


In [ ]:
start = time.time() 
party_details_paidout_anonymised = anonymizer_iban_post(party_details_paidout_anonymised, list_iban_party)
stop = time.time()
print(f'Anonymisation IBAN Post Time {stop - start}s')

col party_identifier_id
Dropping col
Anonymisation IBAN Post Time 9.248136043548584s


In [ ]:
real_data_dict['bd_party_details_paidout'] = party_details_paidout_anonymised

### Anonymisation Multi RelationShip

- bd_account.iban_number = remittance_paidin.account
- bd_account.external_reference_1 = remittance_paidin.debtor_account
- bd_account.iban_number = bd_dd_paidout.pos_id

In [ ]:
"""
sample_data_dict = {}
for table_name, data in real_data_dict.items():
    print(f'table_name : {table_name}, shape: {data.shape}')
    if table_name == 'bd_communication_address':
        pass
    elif table_name == 'rd_fccr_clear_system':
        pass
    else: 
        sample_data_dict[table_name] = data.sample(n = 5000)
"""

"\nsample_data_dict = {}\nfor table_name, data in real_data_dict.items():\n    print(f'table_name : {table_name}, shape: {data.shape}')\n    if table_name == 'bd_communication_address':\n        pass\n    elif table_name == 'rd_fccr_clear_system':\n        pass\n    else: \n        sample_data_dict[table_name] = data.sample(n = 5000)\n"

In [ ]:
real_data_dict['bd_dd_remittance_paidin']['iban_number'] = real_data_dict['bd_dd_remittance_paidin']['account']

In [ ]:
real_data_dict['bd_dd_remittance_paidin']['external_reference_1'] = real_data_dict['bd_dd_remittance_paidin']['debtor_account']

In [ ]:
real_data_dict['bd_dd_paidout']['iban_number'] = real_data_dict['bd_dd_paidout']['pos_id']

In [ ]:
anonymised_dict_multi_1 =  [{'table_name_list': ['bd_account', 'bd_dd_remittance_paidin', 'bd_dd_paidout'], 'col': ['iban_number'], 'category': 'iban_number'}, 
]

In [ ]:
anonymised_dict_multi_2 =  [{'table_name_list': ['bd_account', 'bd_dd_remittance_paidin'], 'col': ['external_reference_1'], 'category': 'iban_number'}, 
]

In [ ]:
for anonym_dict in anonymised_dict_multi_1:
    print(anonym_dict)
    for table_name in anonym_dict['table_name_list']:
        print(table_name)
        start = time.time()
        real_data_dict[table_name] = anonymizer_iban_prepro(real_data_dict[table_name], anonym_dict['col'])
        stop = time.time()
        print(f'Anonymisation {table_name} Time {stop - start}s')

{'table_name_list': ['bd_account', 'bd_dd_remittance_paidin', 'bd_dd_paidout'], 'col': ['iban_number'], 'category': 'iban_number'}
bd_account
col iban_number
Anonymisation bd_account Time 0.141646146774292s
bd_dd_remittance_paidin
col iban_number
Anonymisation bd_dd_remittance_paidin Time 1.1888279914855957s
bd_dd_paidout
col iban_number
Anonymisation bd_dd_paidout Time 0.6270508766174316s


In [ ]:
for anonym_dict in anonymised_dict_multi_2:
    print(anonym_dict)
    for table_name in anonym_dict['table_name_list']:
        start = time.time()
        real_data_dict[table_name] = anonymizer_iban_prepro(real_data_dict[table_name], anonym_dict['col'])
        stop = time.time()
        print(f'Anonymisation {table_name} Time {stop - start}s')

{'table_name_list': ['bd_account', 'bd_dd_remittance_paidin'], 'col': ['external_reference_1'], 'category': 'iban_number'}
col external_reference_1
Anonymisation bd_account Time 0.07944679260253906s
col external_reference_1
Anonymisation bd_dd_remittance_paidin Time 1.6807012557983398s


In [ ]:
start = time.time()
real_data_dict_anonymised, mapping_multi = anonymizer_multi(real_data_dict, anonymised_dict_multi_1)
stop = time.time()
print(f'Anonymisation Time {stop - start}s')

col iban_number
table bd_account
table bd_dd_remittance_paidin
table bd_dd_paidout
Anonymisation Time 959.0857992172241s


In [ ]:
start = time.time()
real_data_dict_anonymised, mapping_multi = anonymizer_multi(real_data_dict, anonymised_dict_multi_2)
print(f'Anonymisation Time {stop - start}s')

col external_reference_1
table bd_account
table bd_dd_remittance_paidin
Anonymisation Time -0.004858970642089844s


In [ ]:
for anonym_dict in anonymised_dict_multi_1:
    print(anonym_dict)
    for table_name in anonym_dict['table_name_list']:
        print(table_name)
        start = time.time()
        real_data_dict[table_name] = anonymizer_iban_post(real_data_dict[table_name], anonym_dict['col'])
        stop = time.time()
        print(f'Anonymisation {table_name} Time {stop - start}s')

{'table_name_list': ['bd_account', 'bd_dd_remittance_paidin', 'bd_dd_paidout'], 'col': ['iban_number'], 'category': 'iban_number'}
bd_account
col iban_number
Dropping col
Anonymisation bd_account Time 0.7435026168823242s
bd_dd_remittance_paidin
col iban_number
Dropping col
Anonymisation bd_dd_remittance_paidin Time 14.988747835159302s
bd_dd_paidout
col iban_number
Dropping col
Anonymisation bd_dd_paidout Time 6.804426670074463s


In [ ]:
for anonym_dict in anonymised_dict_multi_2:
    print(anonym_dict)
    for table_name in anonym_dict['table_name_list']:
        print(table_name)
        start = time.time()
        real_data_dict[table_name] = anonymizer_iban_post(real_data_dict[table_name], anonym_dict['col'])
        stop = time.time()
        print(f'Anonymisation {table_name} Time {stop - start}s')

{'table_name_list': ['bd_account', 'bd_dd_remittance_paidin'], 'col': ['external_reference_1'], 'category': 'iban_number'}
bd_account
col external_reference_1
Dropping col
Anonymisation bd_account Time 0.7885088920593262s
bd_dd_remittance_paidin
col external_reference_1
Dropping col
Anonymisation bd_dd_remittance_paidin Time 17.498265981674194s


### Write Parquet Anonymised Data

In [10]:
tracking_id_anonym = 'gtps_fccr_model_anonym_v2'

In [11]:
anonymised_path_bd_account = f'a495587/original_data/original_{tracking_id_anonym}/int_bd_account/data/bd_account_anonym.parquet'
anonymised_path_bpmainonscreen = f'a495587/original_data/original_{tracking_id_anonym}/int_bd_bpmainonscreen/data/bpmainonscreen_anonym.parquet'
anonymised_path_communication_address = f'a495587/original_data/original_{tracking_id_anonym}/int_bd_communication_address/data/communication_address_anonym.parquet'
anonymised_path_transfer_remittance = f'a495587/original_data/original_{tracking_id_anonym}/int_bd_transfer_remittance/data/bd_remittance_anonym.parquet'
anonymised_path_dd_paidin = f'a495587/original_data/original_{tracking_id_anonym}/int_bd_dd_remittance_paidin/data/dd_remittance_paidin_anonym.parquet'
anonymised_path_party_details_paidin = f'a495587/original_data/original_{tracking_id_anonym}/int_bd_party_details_paidin/data/party_details_anonym.parquet'
anonymised_path_dd_paidout = f'a495587/original_data/original_{tracking_id_anonym}/int_bd_dd_paidout/data/dd_paidout_anonym.parquet'
anonymised_path_party_details_paidout = f'a495587/original_data/original_{tracking_id_anonym}/int_bd_party_details_paidout/data/party_details_anonym.parquet'

In [ ]:
save_dict = {'bd_account': anonymised_path_bd_account, 'bd_bpmainonscreen': anonymised_path_bpmainonscreen, 
             'bd_communication_address': anonymised_path_communication_address, 'bd_transfer_remittance': anonymised_path_transfer_remittance,
             'bd_dd_remittance_paidin': anonymised_path_dd_paidin, 'bd_party_details_paidin': anonymised_path_party_details_paidin,
             'bd_dd_paidout': anonymised_path_dd_paidout, 'bd_party_details_paidout': anonymised_path_party_details_paidout
            }

In [ ]:
for table_name, path in save_dict.items():
    s3_connector.write_parquet(path, real_data_dict[table_name])

### Check Joining conditions between Bd_account & Dd_paidout

In [12]:
test_bd_account = read_parquet(s3_connector, anonymised_path_bd_account)

In [13]:
test_dd_paidout = read_parquet(s3_connector, anonymised_path_dd_paidout)

In [24]:
isin_bd_account = test_bd_account[test_bd_account['iban_number'].isin(test_dd_paidout['pos_id'])]

In [25]:
isin_bd_account.shape

(7761, 94)

In [22]:
isin_dd_paidout = test_dd_paidout[test_dd_paidout['pos_id'].isin(test_bd_account['iban_number'])]

In [23]:
isin_dd_paidout.shape

(1, 97)